# Introduction
This practical work is about creation of elementary models in keras.
While it focuses on image classification, it illustrates many important functionalities of the keras framework and tries to provide the user with good deep-learning development strategies.

## Setup
Before you start, please mount your google drive.
For each exercise, create an output directory to store monitoring data and your model.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# EX1: Lenet-like classifier.

## 1- Build model architecture
The architecture has to be a CLASS called "Lenet_like".

- I can define the width: number of filters in the first layer.
- I can define the depth: number of conv layers.
- I can specify the number of classes: output neurons.

Lenet_like has no input tensor. But it has a \_\_call\_\_ function and can therefore be called on an input later.

The rule to go from depth d to depth d+1, is to reduce the spatial size by a factor of 2 in each direction.

Hidden dense layer will have 512 units.

In [2]:
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, Dense
from keras.models import Model

class Lenet_like:
  
  def __init__(self, width, depth, nb_classes):
    self.width = width
    self.depth = depth
    self.nb_classes = nb_classes

  def __call__(self, input_tensor):
    y = Conv2D(self.width, kernel_size = (3,3), activation='relu', padding='same')(input_tensor)
    y = MaxPooling2D(pool_size=(2,2), padding='same')(y)

    for d in range(self.depth) :
      y = Conv2D(self.width*2*(d+1), kernel_size = (3,3), activation='relu', padding='same')(y)
      y = MaxPooling2D(pool_size=(2,2), padding='same')(y)
    
    y = Dense(512, activation='relu')(y)
    out = Dense(self.nb_classes, activation='softmax')(y)

    model = Model(inputs=input_tensor, outputs=out)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

Using TensorFlow backend.


In [0]:
# Test the class
test = Lenet_like(5,5,2)

## 2- A function to create a model with Lenet_like architecture
I want the model to be able to fit on the following datasets:

- mnist
- fashion-mnist
- cifar-10

Create a function called "make_lenet_model" that take the name of one of these dataset as a str.
It returns a keras Model object.
It should obviously take all arguments to init the Lenet_like architecture.
Arguments other than the dataset might have default values.
I want to be able to monitor the accuracy of the model.

In [0]:
import tensorflow as tf
from keras import datasets

def make_lenet_model(name_dataset, width, depth, nb_classes):

  if str(name_dataset) == 'mnist':
    (x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

  if str(name_dataset) == 'fashion-mnist':
    (x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()
  
  if str(name_dataset) == 'cifar-10':
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

  model = Lenet_like(width, depth, nb_classes)
  input_tensor = len(x_train)
  out = model(input_tensor)

  return out, x_train, y_train, x_test, y_test

## 3- Create a fitting function

I want a function "fit_model_on" with the following arguments:

- dataset: str name of the dataset
- epochs: number of times you fit on the entire training set
- batch_size: number of images to average gradient on

The function must create a Lenet model and fit it following these parameters.
The function should:

- fit the model, obviously
- store the model architecture in a .json file in your output directory
- store the model's weights in a .h5 file in your output directory
- store the fitting metrics loss, validation_loss, accuracy, validation_accuracy under the form of a plot exported in a png file.

Run your fitting function of course.

In [0]:
import json
from matplotlib import pyplot as plt

path_output = '/drive/MyDrive/Colab Notebooks/Big Data/TP7/output/'

def fit_model_on(dataset, epochs, batch_size, width, depth, nb_classes):
  # apply the function make_lenet_model
  model, x_train, y_train, x_test, y_test = make_lenet_model(dataset, width, depth, nb_classes)
  
  # fit the model
  model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

  # store the model architecture
  model_json = model.to_json()
  with open(path_output+'model.json', 'w') as json_file:
    json_file.write(model_json)

  # store the model's weights
  model.save_weights(path_output+'model.h5')

  #store the fitting metrics
  scores = model.evaluate(x_train, y_train, verbose=0)
  names_score = model.metrics_names
  plt.plot(score[2], label = name_score[2])
  plt.plot(score[3], label = name_score[3])
  plt.plot(score[4], label = name_score[4])
  plt.plot(score[5], label = name_score[5])
  plt.title("Metrics of the model on the train")
  plt.legend(bbox_to_anchor=(1,1))
  plt.savefig('metrics.png')

In [6]:
# Run the function
dataset = 'fashion-mnist'
epochs = 10
batch_size = 100
width = 10000
depth = 10
nb_classes = 2

fit_model_on(dataset, epochs, batch_size, width, depth, nb_classes)

4423680/4422102 [==============================] - 1s 0us/step


ValueError: ignored